In [ ]:
# "I certify that the code and data in this assignment were generated independently, using only the tools and resources defined in the course and that I did not receive any external help, coaching or contributions during the production of this work."

In [ ]:
!pip install gym

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import spaces

In [ ]:
from google.colab import widgets
import time

#Part I

In [ ]:
class GridEnvironment(gym.Env):
    metadata = { 'render.modes': [] }
    
    def __init__(self):
        self.observation_space = spaces.Discrete(16) #Intializing a 4x4 grid with 16 states: {s1, s2, s3,....s16}
        self.action_space = spaces.Discrete(4) #Intializing 4 actions:  {0: down, 1: up, 2: right, 3: left}
        self.done = False
        #rewards = {-2, -1, 0, 3, 4, 100} defined below
        
    def reset(self):
  
        self.agent_pos = [0, 0] #start position
        self.goal_pos = [3, 3] #target position (+100)
        self.danger1_pos = [1,1] #first danger position (-1)
        self.danger2_pos = [2,2] #second danger position (-2)
        self.gold1_pos = [2,0] #First positive reward position (+3)
        self.gold2_pos = [3,0] #Second positive reward position (+4)
        self.done = False
        self.state = np.zeros((4,4))
        observation = self.state.flatten()
        return observation
    
    def step(self, action, deterministic = False, stochastic = False):

        #determinstic environment
        if deterministic == True:
          epsilon1, epsilon2 = 1, 1 #With probability 1 Agent chooses given action

        #stochastic environment
        if stochastic == True:
          epsilon1, epsilon2 = 0.7, 0.8 #Transistion probabilites of 0.7 and 0.8 
          #Σ(p(s', r/s, down) = 0.7 + 0.3 = 1
          #Σ(p(s', r/s, right) = 0.8 + 0.2 = 1 

        #Actions
        if action == 0: #down
            rand_num1 = np.random.random()
            if epsilon1 >= rand_num1:  
              self.agent_pos[0] += 1   
            else: #For all states in stochastic environment when Down action is choosen: Agent chooses down state with a transition probability of 0.7 and up state with 0.3 transition probaility
              self.agent_pos[0] -= 1
              # print("Up state is choosen with 0.3 probability instead of Down")

        if action == 1: #up
            self.agent_pos[0] -= 1

        if action == 2: #right
            rand_num2 = np.random.random()
            if epsilon2 >= rand_num2:  
              self.agent_pos[1] += 1        
            else:#For all states in stochastic environment when Right action is choosen: Agent chooses right state with a transition probability of 0.8 and left state with 0.2 transition probaility
              self.agent_pos[1] -= 1
              # print("Left state is choosen with 0.2 probability instead of Right")

        if action == 3: #left
            self.agent_pos[1] -= 1
          
        self.agent_pos = np.clip(self.agent_pos, 0, 3) #ensuring agent doesn't go out of the grid #One of the ways to ensure safety in the environment
        self.state = np.zeros((4,4))

        observation = self.state.flatten()
        
        reward = 0 #Intializing reward to zero
        if (self.agent_pos == self.goal_pos).all():
            reward = 100 #A reward of 5 if it reaches target position
            self.done = True

        #Rewards structure
        elif (self.agent_pos == self.danger1_pos).all():
            reward = -1 #A negative reward -1 if it enters 1st danger position

        elif (self.agent_pos == self.danger2_pos).all():
            reward = -2 #A negative reward of -2 if it enters 2nd danger position

        elif (self.agent_pos == self.gold1_pos).all():
            reward = 3 #A reward of +3 at [2,0]

        elif (self.agent_pos == self.gold2_pos).all():
            reward = 4 #A reward of +3 at [3,0]

        return reward, self.agent_pos, self.done
        
    def render(self):
        plt.imshow(self.state)

In [ ]:
class RandomAgent: #Definig the Random agent class
    def __init__(self, env):
        self.env = env
        self.observation_space = env.observation_space
        self.action_space = env.action_space

    def step(self, observation): #Random agent chooses random action every time we call step method
        return np.random.choice(self.action_space.n)

In [ ]:
#deterministic environment
env = GridEnvironment()
agent = RandomAgent(env)#creating a random agent to explore the given environments 
obs = env.reset()#resets the environment to its initial configuration

done = False
timestep = 0
epsilon = 0.8 # For all states in deterministic environment p(s', r/s, a) = {0, 1}: Either action taken or No action taken
while timestep < 30:#running for 30 timesteps

    action = agent.step(obs)
    timestep += 1

    # deterministic
    rand_num = np.random.random()
    if epsilon > rand_num: #Choosing an action in deterministic environment
      reward, pos, done = env.step(action, deterministic = True)#calling step method to randomly choose an action
      print("Timestep: {}, Reward: {}".format(timestep, reward))
    else:
      print("Timestep: {}, No action taken".format(timestep)) #Random agent doesn't choose any action in deterministic environment
    
    if done == True:
      print("Target Reached!!") #End the program if Random Agent reaches the target position
      break

Timestep: 1, Reward: 0
Timestep: 2, Reward: 3
Timestep: 3, No action taken
Timestep: 4, Reward: 0
Timestep: 5, Reward: 3
Timestep: 6, No action taken
Timestep: 7, No action taken
Timestep: 8, Reward: 3
Timestep: 9, Reward: 4
Timestep: 10, Reward: 0
Timestep: 11, Reward: 0
Timestep: 12, Reward: 0
Timestep: 13, Reward: 0
Timestep: 14, Reward: -1
Timestep: 15, No action taken
Timestep: 16, No action taken
Timestep: 17, Reward: 0
Timestep: 18, Reward: 3
Timestep: 19, Reward: 0
Timestep: 20, Reward: -2
Timestep: 21, Reward: 0
Timestep: 22, Reward: 100
Target Reached!!


In [ ]:
#stochastic environment
env = GridEnvironment()
agent = RandomAgent(env)#creating a random agent to explore the given environments 
obs = env.reset()#resets the environment to its initial configuration


timestep = 0
while timestep < 30: #running for 30 timesteps

    action = agent.step(obs)#calling step method to randomly choose an action
    timestep += 1

    #stochastic
    reward, pos, done = env.step(action, stochastic = True)#Choosing an action in stochastic environment
    print("Timestep: {}, Reward: {}".format(timestep, reward)) #Tracks the timestep and reward after every episode

    if done == True:
      print("Target Reached!!") #End the program if Random Agent reaches the target position
      break

Timestep: 1, Reward: 0
Timestep: 2, Reward: 0
Timestep: 3, Reward: 0
Timestep: 4, Reward: 0
Timestep: 5, Reward: 0
Timestep: 6, Reward: 0
Timestep: 7, Reward: 0
Timestep: 8, Reward: 0
Timestep: 9, Reward: 0
Timestep: 10, Reward: 0
Timestep: 11, Reward: 0
Timestep: 12, Reward: 0
Timestep: 13, Reward: 0
Timestep: 14, Reward: 0
Timestep: 15, Reward: 0
Timestep: 16, Reward: -1
Timestep: 17, Reward: 0
Timestep: 18, Reward: -1
Timestep: 19, Reward: 0
Timestep: 20, Reward: 3
Timestep: 21, Reward: 4
Timestep: 22, Reward: 0
Timestep: 23, Reward: 0
Timestep: 24, Reward: 0
Timestep: 25, Reward: 100
Target Reached!!
